In [12]:
import random
import numpy as np
import cv2

In [17]:

def get_matches(img1, img2, t):#定义一个函数 get_matches()，输入两张图片和一个阈值 t
  sift=cv2.SIFT_create(t)#使用 OpenCV 中的 SIFT 模块来创建一个 SIFT 特征检测器，并指定阈值 t
  k1, d1 = sift.detectAndCompute(img1, None)#使用 SIFT 特征检测器对两张图片中的特征点进行检测，并获取它们的描述符
  k2, d2 = sift.detectAndCompute(img2, None)
  matches = []
  pt1 = 0
  pt2 = 0
    #在两张图片中匹配特征点。对于每个第一张图片中的特征点 d1[i]，在第二张图片的所有特征点 d2 中寻找与它最接近的点，然后将这两个点的坐标加入到 matches 列表中
  for i in range(len(d1)):
      least = 100
      for j in range(len(d2)):
          n = np.linalg.norm(d1[i]-d2[j]) #Compute Euclidean Distance
          if n < least: 
              least = n
              pt1 = i
              pt2 = j
      matches.append([k1[pt1].pt,k2[pt2].pt])
  return matches
def findHomography(matches):#定义一个函数 findHomography()，输入匹配列表 matches
    maxliner = []
    finalH = None
    #初始化变量 maxliner 和 finalH
    # r=np.matrix([[0],[0],[0],[0],[0],[0],[0],[0],[1]])

    for i in range(1000):
        #从匹配列表 matches 中随机选择四个点
        c = matches[random.randrange(0, len(matches))]
        c1 = matches[random.randrange(0, len(matches))]
        c2 = matches[random.randrange(0, len(matches))]
        c3 = matches[random.randrange(0, len(matches))]
        #将这4个匹配点对应的坐标转化为一个2x9的矩阵M。其中，每个2x9的小矩阵分别由下面两个3x3的小矩阵叠加而成：
        """
         [-x1 -y1 -1 0 0 0 x1*y1 x1*y2 x1]
 
            [0 0 0 -x1 -y1 -1 y1*y2 y2*y2 y2]
 
            其中，(x1, y1)和(x2, y2)分别为一对匹配点的在两幅图像中的坐标。
        """
        # Stack four arrays of 2x9 size
        p1 = np.array([-c[0][0], -c[0][1], -1, 0, 0, 0, c[0][0] * c[1][0], c[0][1] * c[1][0], c[1][0]])
        p1_1 = np.array([0, 0, 0, -c[0][0], -c[0][1], -1, c[0][0] * c[1][1], c[0][1] * c[1][1], c[1][1]])
        y = np.vstack((p1, p1_1))

        p2 = np.array([-c1[0][0], -c1[0][1], -1, 0, 0, 0, c1[0][0] * c1[1][0], c1[0][1] * c1[1][0], c1[1][0]])
        p2_1 = np.array([0, 0, 0, -c1[0][0], -c1[0][1], -1, c1[0][0] * c1[1][1], c1[0][1] * c1[1][1], c1[1][1]])
        y_1 = np.vstack((p2, p2_1))

        p3 = np.array([-c2[0][0], -c2[0][1], -1, 0, 0, 0, c2[0][0] * c2[1][0], c2[0][1] * c2[1][0], c2[1][0]])
        p3_1 = np.array([0, 0, 0, -c2[0][0], -c2[0][1], -1, c2[0][0] * c2[1][1], c2[0][1] * c2[1][1], c2[1][1]])
        y_2 = np.vstack((p3, p3_1))

        p4 = np.array([-c3[0][0], -c3[0][1], -1, 0, 0, 0, c3[0][0] * c3[1][0], c3[0][1] * c3[1][0], c3[1][0]])
        p4_1 = np.array([0, 0, 0, -c3[0][0], -c3[0][1], -1, c3[0][0] * c3[1][1], c3[0][1] * c3[1][1], c3[1][1]])
        y_3 = np.vstack((p4, p4_1))

        y_4 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1])
        M = np.vstack((y, y_1, y_2, y_3))
        # M=np.matrix(M)
        #对矩阵M进行奇异值分解（SVD），并取SVD分解结果中的第8列（编号从0开始）作为单应性矩阵H。注意，这里的H是一个3x3的矩阵，不是一个向量。
        u, s, v = np.linalg.svd(M)
        H = np.reshape(v[8], (3, 3))  # Get the 3x3 homography matrix

        H = (1 / H.item(8)) * H
        #对矩阵H进行归一化处理，即将H中所有元素除以其最后一个元素
        aliner = []
        """
        于所有匹配点对，利用计算出的单应性矩阵H进行变换，
        将第一幅图像中的特征点坐标转换为在第二幅图像中的预测坐标，然后计算预测坐标与实际匹配点在第二幅图像中的坐标之间的距离。
        若距离小于一个阈值（这里的阈值为10），则认为该匹配点对是一致的，并将其加入列表aliner中。
        """
        for i in range(len(matches)):
            coordinates_H = matches[i]
            original_pt = np.transpose(np.matrix([coordinates_H[0][0], coordinates_H[0][1], 1]))
            calculated_pt = np.dot(H, original_pt)
            calculated_pt = (1 / calculated_pt.item(2)) * calculated_pt
            p2 = np.transpose(np.matrix([coordinates_H[1][0], coordinates_H[1][1], 1]))
            distance = p2 - calculated_pt
            distance = np.linalg.norm(distance)
            if distance < 10:
                aliner.append(coordinates_H)
        #如果列表aliner中的匹配点对数量大于列表maxliner中的数量，则将aliner赋值给maxliner，将H的逆矩阵赋值给homography，并将homography进行归一化处理。
        if len(aliner) > len(maxliner):  # Keep Best Homography Matrix and Normalize
            maxliner = aliner
            homography = H
            homography = np.linalg.inv(homography)
            homography = (1 / homography[2][
                2]) * homography  # Normalize the homography matrix by dividing with last element
        #最后，函数将最大列表中的匹配点对用于计算最终的单应性矩阵，并将该矩阵返回
    return homography



In [18]:
img1 = cv2.imread('image1.jpg')
img2 = cv2.imread('image2.jpg')
print('Getting matches...')
matches = get_matches(img1, img2, 3000)	#Modify the number of features depending on the size/type of your images
print('Finding Homography...')
homography = findHomography(matches)
print(homography)

Getting matches...
Finding Homography...
[[ 7.83627094e-01  1.60042191e-01  3.05716090e+02]
 [-1.11962314e-01  9.30269783e-01 -6.65573588e+01]
 [-1.97235708e-04 -3.54439847e-05  1.00000000e+00]]
